In [1]:
#IMPORTAMOS LIBRERÍA PARA CONECTAR MYSQL-PYTHON
import mysql.connector
#IMPORTAMOS LIBRERÍA PARA INTERACTUAR CON SERVICIOS WEB Y CONSUMIR APIs
import requests
#IMPORTAMOS LIBRERÍA PARA ACCEDER A VARIABLES Y FUNCIONES ESPECÍFICAS DEL INTERPRETE DE PYTHON
import sys

In [2]:
#DEFINIMOS COLORES PARA LOS TEXTOS CON CODIGO ANSI ESCAPE EN FORMATO OCTAL Y DE COLORES EN PRIMER PLANO
#https://gist.github.com/fnky/458719343aabd01cfb17a3a4f7296797
class Colors:
    BLACK = '\033[90m'
    WHITE= '\033[97m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    CYAN = '\033[36m'
    RED = '\033[91m'
    YELLOW = '\033[93m'
    ENDC = '\033[0m'

In [3]:
#FUNCIÓN PARA MOSTRAR AVISOS AL USUARIO Y CONTROLAR ERRORES DE VALORES DE ENTRADA O  ERORES DE CARGA DE DATOS 
def warning(message):
    print(f"{Colors.RED}* ATENCIÓN *{Colors.ENDC} {message}")

In [4]:
#CREAMOS Y DEVOLVEMOS UNA FUNCIÓN DE CONEXIÓN A UNA BASE DE DATOS CON PARÁMETROS ESPECÍFICOS EN MYSQL
def create_connection(host, user, password, database):
    mydb = mysql.connector.connect(
        host=host,
        user=user,#Usuario
        password=password,#contraseña
        database=database #esquema
    )
    return mydb

In [5]:
#GUARDAMOS EN LA VARIABLE CONEXION LA FUNCIÓN PREVIAMENTE CREADA PARA MANEJARLA MÁS FÁCILMENTE
conexion = create_connection("dayhoff.inf.um.es", "alubio08", "alubio08PW-", "alubio08")
print(conexion)

In [6]:
#LO PONGO COMO COMENTARIO PARA PODER EJECUTAR TODAS LAS CELDAS Y QUE FUNCIONE LA FUNCIÓN MAIN
#conexion.close()

In [7]:
#CONSULTA 1 -> Resultado agrupado de los pacientes por las propiedades sexo y rango de edad.
def buscar_por_muestra_sexo_edad():
    #Abrimos cursor
    mycursor = conexion.cursor()
    #Vamos a seleccionar las filas donde tengamos el nombre de propiedad mujer u hombre (como sexo) y lo mismo para
    #los distintos rangos de edad, además de contar el número de muestras. Esto vamos tener que hacer join con las tablas
    #correspondientes agrupando finalmente por cada nombre de sexo y cada rango de edad junto su numero de muestras.
    #DESTACAMOS -> solo va a moestrar resultados de aquellos grupos para los que tengamos especificado tanto el tipo de sexo
    #como su rango de edad, si alguno de esos tipos no esta especificado no lo mostrará.
    mycursor.execute("""SELECT p_sexo.nombre AS sexo, p_rango_edad.nombre AS rango_de_edad, COUNT(DISTINCT m.codigo) AS n_muestras
                        FROM b3b2_muestra m INNER JOIN b3b2_muestra_propiedad mp_sexo ON m.codigo = mp_sexo.muestra
                        INNER JOIN b3b2_propiedad p_sexo ON mp_sexo.propiedad = p_sexo.id
                        INNER JOIN b3b2_muestra_propiedad mp_rango_edad ON m.codigo = mp_rango_edad.muestra
                        INNER JOIN b3b2_propiedad p_rango_edad ON mp_rango_edad.propiedad = p_rango_edad.id
                        WHERE p_sexo.tipo = 'sexo' AND p_rango_edad.tipo = 'rango de edad'
                        GROUP BY p_sexo.nombre, p_rango_edad.nombre
                        ORDER BY p_sexo.nombre, p_rango_edad.nombre;""")
    myresult = mycursor.fetchall() #Le indicamos que nos cargue todos los valores recuperados
    #Verificamos que en la consulta haya algún resultado
    if not myresult:
        print("No hay resultados para la consulta.")
    #Organizamos los resultados guardados en la variable myresult en una tabla
    else:
        print(f"{Colors.GREEN}Resultado agrupado de los pacientes por las propiedades sexo y rango de edad:{Colors.ENDC}")
        print("{:<10}  {:<10}  {:<15}".format("Sexo", "Rango Edad", "N Muestras"))
        print("-" * 40)
        for resultado in myresult:
            print("{:<10}  {:<10}  {:<15}".format(resultado[0], resultado[1], resultado[2]))
    #Cada vez que hemos terminado una operación con la base de datos cerramos el cursor
    mycursor.close()

In [8]:
#CONSULTA 2 -> Número de genes que participan en la rutas que contenta la palabra 'platelet'
def buscar_por_genes_rutas_platelet():
    #Abrimos cursor 
    mycursor = conexion.cursor()
    #Vamos a contar todos los genes distintos de la tabla donde tenemos genes y pathways
    #donde coincida que en la descripcion del pathway (join a tabla pathway) donde tiene registrada 
    #su descripcion aparezca la palabra "platelet"
    mycursor.execute("""SELECT COUNT(DISTINCT pg.gen) AS numero_genes
                        FROM b3b2_gen_pathway pg
                        INNER JOIN b3b2_pathway p ON pg.pathway = p.codigo
                        WHERE p.descripcion LIKE '%platelet%';""")
    myresult = mycursor.fetchone() #Le indicamos que nos cargue el único valor recuperado
    #Verificamos que hay resultados en la consulta 
    if not myresult:
        print("No hay genes que tengan la palabra platelet incluida en la descripcion de su ruta.")
    else:
        #Mostramos el primer y único resultado de la variable myresult
        print(f"{Colors.CYAN}Número de genes que participan en la rutas que contenta la palabra 'platelet' --> {Colors.ENDC} {myresult[0]}")
    #Cerramos cursor
    mycursor.close()

In [9]:
#CONSULTA 3 -> Muestras de las que no tenemos resultados de expresión
def buscar_muestras_noexpresion():
    #Abrimos cursor
    mycursor = conexion.cursor()
    #Seleccionamos todos los códigos de la tabla muestra que no estan presentes en la tabla de expresion
    mycursor.execute("""SELECT m.codigo FROM b3b2_muestra m
                        WHERE m.codigo NOT IN (SELECT DISTINCT muestra FROM b3b2_expresion);""")
    myresult = mycursor.fetchall() #Le indicamos que nos cargue todos los valores recuperados
    #Verificamos que hay resultados en la consulta 
    if not myresult:
        print("No hay muestras sin resultados de expresión.")
    else:
        #Mostrados cada código de la muestra para la que no tenemos resultados de expresión
        print(f"{Colors.YELLOW}Muestras de las que no tenemos resultados de expresión:{Colors.ENDC}")
        for muestra in myresult:
            print(muestra[0])
    #Cerramos cursor
    mycursor.close()

In [10]:
#CONSULTA 4 -> Media de expresión de los genes secuenciados. Devolverá el gene_symbol y la media de expresión global.
def buscar_symbol_media_expresion():
    #Abrimos cursor 
    mycursor = conexion.cursor()
    #Seleccionamos columna gene_symbol de tabla de genes donde tendremos columna con el id de ensembl
    #estos ensembl los tendremos en la tabla expresion y vamos a seleccionar la media de las expresiones (average)
    #join entre ensembl de ambas tablas y agrupamos por gene_symbol (solo presente en la tabla gen).
    mycursor.execute("""SELECT g.gene_symbol, AVG(e.expresion) AS media_expresion
                        FROM b3b2_gen g
                        INNER JOIN b3b2_expresion e ON g.gene_ensembl = e.gen
                        GROUP BY g.gene_symbol;""")
    myresult = mycursor.fetchall()  #Le indicamos que nos cargue todos los valores recuperados
    #Verificamos que hay resultados en la consulta
    if not myresult:
        print("No hay resultados para la consulta.")
    else:
        #Mostramos cada symbol junto su media de expresión a modo de tabla
        print(f"{Colors.GREEN}Resumen genes y su media de expresión:{Colors.ENDC}")
        print("{:<20}  {:<15}".format("Gen Symbol", "Media Expresión"))
        print("-" * 40)
        for gen in myresult:
            print("{:<20}  {:<15.2f}".format(gen[0], gen[1]))
    #Cerramos cursor
    mycursor.close()

In [11]:
#CONSULTA 5 -> Número de pathways de los genes que tienen media de expresión mayor de 25
def buscar_pathways_genes_media_expresion():
    #Abrimos cursor 
    mycursor = conexion.cursor()
    #Consulta anidada donde vamos a seleccionar el codigo de pathway en el que los genes asociados
    #a esas rutas tengan una media de expresión superior a 25 (seleccionando rutas distintas solamente),
    #ya que algunos genes están en mas de una ruta.
    mycursor.execute("""SELECT COUNT(DISTINCT gp.pathway) AS numero_pathways
                        FROM b3b2_gen g
                        INNER JOIN b3b2_gen_pathway gp ON g.gene_ensembl = gp.gen
                        INNER JOIN b3b2_pathway p ON gp.pathway = p.codigo
                        WHERE g.gene_ensembl IN (SELECT e.gen FROM b3b2_expresion e
                        GROUP BY e.gen HAVING AVG(e.expresion) > 25);""")
    myresult = mycursor.fetchone() #Le indicamos que nos cargue el único valor consultado
    #Verificamos que hay resultados en la consulta 
    if not myresult:
        print("No hay muestras sin resultados de expresión.")
    else:
        #Mostramos el único resultado obtenido  en la variable myresult
        print(f"{Colors.CYAN}Número de pathways de los genes que tienen media de expresión mayor de 25 --> {Colors.ENDC} {myresult[0]}") 
    #Cerramos cursor
    mycursor.close()

In [12]:
#CONSULTA 6 -> Muestras de las que no tenemos alguna propiedad (o sexo o rango de edad)
def buscar_muestras_no_propiedad():
    #Abrimos cursor
    mycursor = conexion.cursor()
    #la consulta busca códigos de muestra distintos que no tienen propiedades asociadas 
    #de tipo 'sexo' o 'rango de edad', utilizando LEFT JOIN para incluir todas las muestras, 
    #incluso aquellas que no tienen coincidencias en las tablas de propiedades.
    mycursor.execute("""SELECT DISTINCT m.codigo FROM b3b2_muestra m
                        LEFT JOIN b3b2_muestra_propiedad mp_sexo ON m.codigo = mp_sexo.muestra
                        AND mp_sexo.propiedad IN (SELECT id FROM b3b2_propiedad WHERE tipo = 'sexo')
                        LEFT JOIN b3b2_muestra_propiedad mp_rango_edad ON m.codigo = mp_rango_edad.muestra
                        AND mp_rango_edad.propiedad IN (SELECT id FROM b3b2_propiedad WHERE tipo = 'rango de edad')
                        WHERE mp_sexo.muestra IS NULL OR mp_rango_edad.muestra IS NULL;""")
    myresult = mycursor.fetchall() #Le indicamos que nos cargue todos los valores recuperados
    #Verificamos que hay resultados en la consulta 
    if not myresult:
        print("No hay muestras sin resultados de expresión.")
    else:
        #Mostramos el código de muestra para la que no tenemos alguna propiedad en una lista
        muestra_list = [muestra[0] for muestra in myresult]
        print(f"{Colors.YELLOW}Muestras de las que no tenemos alguna propiedad (o sexo o rango de edad): {Colors.ENDC} {muestra_list}")
    #Cerramos cursor
    mycursor.close()

In [13]:
#CONSULTA 7 -> Número de muestras por run
def buscar_muestras_por_run():
    #Abrimos cursor
    mycursor = conexion.cursor()
    #Vamos a contar el número de muestras y las vamos a agrupar por run, seleccionando el nombre
    #de cada run para mostrar el número de muestras que posee asociado cada uno
    mycursor.execute("""SELECT r.nombre AS run, COUNT(mr.muestra) AS numero_muestras
                        FROM beb2_run r
                        INNER JOIN beb2_muestra_run mr ON r.nombre = mr.run
                        GROUP BY r.nombre;""")
    myresult = mycursor.fetchall()  #Le indicamos que nos cargue todos los valores recuperados
    #Verificamos que hay resultados en la consulta
    if not myresult:
        print("No hay resultados para la consulta.")
    else:
        #Mostramos el número de muestras para cada run en una tabla 
        print(f"{Colors.GREEN}Resumen run y número de muestras:{Colors.ENDC}")
        print("{:<20}  {:<15}".format("Run", "N Muestras"))
        print("-" * 40)
        for run in myresult:
            print("{:<20}  {:<15.2f}".format(run[0], run[1]))
    #Cerramos cursor
    mycursor.close()

In [14]:
#CONSULTA 8 -> Consulta que devuelva los genes cuyo log2foldchange sea mayor de 1 en valor absoluto entre muestras con sexo hombre y muestras con sexo mujer
def buscar_log2foldchange():
    #Abrimos cursor
    mycursor = conexion.cursor()
    #Esta consulta anidada realiza cálculos de media de expresión para hombres y mujeres 
    #por separado, y luego calcula el logaritmo en base 2 del cambio de expresión entre ambos
    #sexos para cada gen. La consulta principal selecciona genes cuyo valor absoluto del logaritmo
    #en base 2 del cambio de expresión es mayor que 1 en valor absoluto.
    mycursor.execute("""SELECT g.gene_symbol, LOG2(media_expresion_hombre / media_expresion_mujer) AS log2foldchange
                        FROM b3b2_gen g
                        INNER JOIN (
                                    SELECT e.gen, AVG(e.expresion) AS media_expresion_hombre
                                    FROM b3b2_expresion e
                                    INNER JOIN b3b2_muestra_propiedad mp ON e.muestra = mp.muestra
                                    INNER JOIN b3b2_propiedad p ON mp.propiedad = p.id
                                    WHERE p.tipo = 'sexo' AND p.nombre = 'hombre'
                                    GROUP BY e.gen
                                   ) AS media_expresion_h ON g.gene_ensembl = media_expresion_h.gen
                        INNER JOIN (
                                    SELECT e.gen, AVG(e.expresion) AS media_expresion_mujer
                                    FROM b3b2_expresion e
                                    INNER JOIN b3b2_muestra_propiedad mp ON e.muestra = mp.muestra
                                    INNER JOIN b3b2_propiedad p ON mp.propiedad = p.id
                                    WHERE p.tipo = 'sexo' AND p.nombre = 'mujer'
                                    GROUP BY e.gen
                                   ) AS media_expresion_m ON g.gene_ensembl = media_expresion_m.gen
                        HAVING ABS(log2foldchange) > 1;""")
    myresult = mycursor.fetchall()  #Le indicamos que nos cargue todos los valores recuperados
    #Verificamos que hay resultados en la consulta
    if not myresult:
        print("No hay resultados para la consulta.")
    else:
        #Mostramos el symbol de los genes y su log2foldchange en una tabla 
        print(f"{Colors.GREEN}Resumen genes y su media de expresión:{Colors.ENDC}")
        print("{:<20}  {:<15}".format("Gen Symbol", "Log2FoldChange"))
        print("-" * 40)
        for gen in myresult:
            print("{:<20}  {:<15.2f}".format(gen[0], gen[1]))
    #Cerramos cursor
    mycursor.close()

In [15]:
#FUNCION PRINCIPAL DE MENÚ DE OPCIONES QUE TIENE EL USUARIO PARA QUE ELIJA QUE QUIERE VER 
def main():
    #INCLUIMOS CONEXION DENTRO DE LA FUNCIÓN PRINCIPAL 
    conexion = create_connection("dayhoff.inf.um.es", "alubio08", "alubio08PW-", "alubio08")
    #MENÚ PRINCIPAL DEL PROGRAMA CON 9 OPCIONES PARA EL USUARIO DONDE CADA OPCIÓN
    #SERÁ ARGUMENTO DE UNA FUNCIÓN ANTERIOREMENTE DEFINIDA Y LA ÚLTIMA CORTA EL BUCLE PARA SALIR
    #CONTROLAMOS POSIBLES ERRORES MOSTRANDO UN AVISO CUANDO NO SE SELECCIONA NINGUNA DE LAS OPCIONES
    while True:
        print(f"{Colors.BLUE}\nCONSULTAS BASE DE DATOS RNA-SEQ{Colors.ENDC}")
        print(f"{Colors.BLACK}---------------------------------------------------------{Colors.ENDC}")
        print("CONSULTA 1. Agrupar los pacientes por las propiedades sexo y rango de edad")
        print("CONSULTA 2. Numerar de genes que participan en la rutas que contenta la palabra 'platelet'")
        print("CONSULTA 3. Listar muestras de las que no tenemos resultados de expresión")
        print("CONSULTA 4. Listar media de expresión de los genes secuenciados")
        print("CONSULTA 5. Numerar pathways de los genes que tienen media de expresión mayor de 25")
        print("CONSULTA 6. Listar muestras de las que no tenemos alguna propiedad (o sexo o rango de edad)")
        print("CONSULTA 7. Mostrar el número de muestras por run")
        print("CONSULTA 8. Mostrar genes cuyo logfoldchange (M sexo H/M sexo M) sea mayor de 1 en valor absoluto")
        print("Salir del programa (Pulse opcion 9)")
        opcion = input("\nIndique la opción que desea ejecutar (1, 2, 3, 4, 5, 6, 7, 8, 9): ")
        if opcion == "":
            warning("La opción indicada no está disponible. Pruebe de nuevo:\n")
        elif opcion == 0:
            warning("La opción indicada no está disponible. Pruebe de nuevo:\n")
        elif "." in opcion:
            warning("La opción indicada no está disponible. Pruebe de nuevo:\n")
        elif opcion == "1":
            buscar_por_muestra_sexo_edad()
        elif opcion == "2":
            buscar_por_genes_rutas_platelet()    
        elif opcion == "3":
            buscar_muestras_noexpresion()                                         
        elif opcion == "4":
            buscar_symbol_media_expresion()
        elif opcion == "5":
            buscar_pathways_genes_media_expresion()
        elif opcion == "6":
            buscar_muestras_no_propiedad()
        elif opcion == "7":
            buscar_muestras_por_run() 
        elif opcion == "8":
            buscar_log2foldchange()
        #SI SE MARCA LA OPCIÓN 9 ROMPEMOS BUCLE DE MENÚ PRINCIPAL Y SALIMOS DEL PROGRAMA
        #Y LO MOSTRAMOS CON EL ÚLTIMO PRINT
        elif opcion == "9":
            break
        #EN CASO DE QUE EL USUARIO COMETA ALGÚN TIPO DE ERROR SE MOSTRARÁ UN AVISO
        #Y SE LE DARÁ LA OPORTUNIDAD DE VOLVER A INTENTARLO
        else:
            warning("La opción indicada no está disponible")
                
    print(f"{Colors.CYAN}\nEl programa ha finalizado correctamente\n{Colors.ENDC}")
#PRIMERA FUNCIÓN EN INICIALIZARSE SERÁ LA FUNCIÓN MAIN    
if __name__ == "__main__":
    main()


CONSULTAS BASE DE DATOS RNA-SEQ
---------------------------------------------------------
CONSULTA 1. Agrupar los pacientes por las propiedades sexo y rango de edad
CONSULTA 2. Numerar de genes que participan en la rutas que contenta la palabra 'platelet'
CONSULTA 3. Listar muestras de las que no tenemos resultados de expresión
CONSULTA 4. Listar media de expresión de los genes secuenciados
CONSULTA 5. Numerar pathways de los genes que tienen media de expresión mayor de 25
CONSULTA 6. Listar muestras de las que no tenemos alguna propiedad (o sexo o rango de edad)
CONSULTA 7. Mostrar el número de muestras por run
CONSULTA 8. Mostrar genes cuyo logfoldchange (M sexo H/M sexo M) sea mayor de 1 en valor absoluto
Salir del programa (Pulse opcion 9)



Indique la opción que desea ejecutar (1, 2, 3, 4, 5, 6, 7, 8, 9):  1


Resultado agrupado de los pacientes por las propiedades sexo y rango de edad:
Sexo        Rango Edad  N Muestras     
----------------------------------------
Hombre      >70         6              
Hombre      18-45       7              
Mujer       <18         7              
Mujer       45-70       7              

CONSULTAS BASE DE DATOS RNA-SEQ
---------------------------------------------------------
CONSULTA 1. Agrupar los pacientes por las propiedades sexo y rango de edad
CONSULTA 2. Numerar de genes que participan en la rutas que contenta la palabra 'platelet'
CONSULTA 3. Listar muestras de las que no tenemos resultados de expresión
CONSULTA 4. Listar media de expresión de los genes secuenciados
CONSULTA 5. Numerar pathways de los genes que tienen media de expresión mayor de 25
CONSULTA 6. Listar muestras de las que no tenemos alguna propiedad (o sexo o rango de edad)
CONSULTA 7. Mostrar el número de muestras por run
CONSULTA 8. Mostrar genes cuyo logfoldchange (M sexo H/M sex


Indique la opción que desea ejecutar (1, 2, 3, 4, 5, 6, 7, 8, 9):  2


Número de genes que participan en la rutas que contenta la palabra 'platelet' -->  23

CONSULTAS BASE DE DATOS RNA-SEQ
---------------------------------------------------------
CONSULTA 1. Agrupar los pacientes por las propiedades sexo y rango de edad
CONSULTA 2. Numerar de genes que participan en la rutas que contenta la palabra 'platelet'
CONSULTA 3. Listar muestras de las que no tenemos resultados de expresión
CONSULTA 4. Listar media de expresión de los genes secuenciados
CONSULTA 5. Numerar pathways de los genes que tienen media de expresión mayor de 25
CONSULTA 6. Listar muestras de las que no tenemos alguna propiedad (o sexo o rango de edad)
CONSULTA 7. Mostrar el número de muestras por run
CONSULTA 8. Mostrar genes cuyo logfoldchange (M sexo H/M sexo M) sea mayor de 1 en valor absoluto
Salir del programa (Pulse opcion 9)


...

* ATENCIÓN * La opción indicada no está disponible

CONSULTAS BASE DE DATOS RNA-SEQ
---------------------------------------------------------
CONSULTA 1. Agrupar los pacientes por las propiedades sexo y rango de edad
CONSULTA 2. Numerar de genes que participan en la rutas que contenta la palabra 'platelet'
CONSULTA 3. Listar muestras de las que no tenemos resultados de expresión
CONSULTA 4. Listar media de expresión de los genes secuenciados
CONSULTA 5. Numerar pathways de los genes que tienen media de expresión mayor de 25
CONSULTA 6. Listar muestras de las que no tenemos alguna propiedad (o sexo o rango de edad)
CONSULTA 7. Mostrar el número de muestras por run
CONSULTA 8. Mostrar genes cuyo logfoldchange (M sexo H/M sexo M) sea mayor de 1 en valor absoluto
Salir del programa (Pulse opcion 9)



Indique la opción que desea ejecutar (1, 2, 3, 4, 5, 6, 7, 8, 9):  4566


* ATENCIÓN * La opción indicada no está disponible

CONSULTAS BASE DE DATOS RNA-SEQ
---------------------------------------------------------
CONSULTA 1. Agrupar los pacientes por las propiedades sexo y rango de edad
CONSULTA 2. Numerar de genes que participan en la rutas que contenta la palabra 'platelet'
CONSULTA 3. Listar muestras de las que no tenemos resultados de expresión
CONSULTA 4. Listar media de expresión de los genes secuenciados
CONSULTA 5. Numerar pathways de los genes que tienen media de expresión mayor de 25
CONSULTA 6. Listar muestras de las que no tenemos alguna propiedad (o sexo o rango de edad)
CONSULTA 7. Mostrar el número de muestras por run
CONSULTA 8. Mostrar genes cuyo logfoldchange (M sexo H/M sexo M) sea mayor de 1 en valor absoluto
Salir del programa (Pulse opcion 9)



Indique la opción que desea ejecutar (1, 2, 3, 4, 5, 6, 7, 8, 9):  9



El programa ha finalizado correctamente

